In [14]:
import pandas as pd
import numpy as np
import Levenshtein
import re
import unidecode 
import json

data = pd.read_excel('../data/Banco de Súmulas.xlsx')

In [15]:
# get unique classes
classes = data['Classe Vela'].unique()
print(classes)

['49er' 'Ilca 6' 'IQFOIL Fem.' 'Formula Kite Masc.' 'IQFOIL Masc.'
 '49erFX' 'Formula Kite Fem.' 'Nacra 17' 'Ilca 7' 470.0]


In [16]:
# keep only pair classes
classes = [470, '49er', '49erFX', 'Nacra 17']
data = data[data['Classe Vela'].isin(classes)]
len(data)

48836

In [17]:
# treating strings

# unidecode
data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)

# replace more than one space by one space
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')

# substituir [;] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r';', r', ')

# substituir [ / ] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r' / ', r', ')

for index, row in data.iterrows():
    if re.match(r'^([A-Z]{2,})', row['Nome Competidor']):
        # ERTHAL Carol LAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,}) ([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])
                
        # ERTHAL CarolLAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,})([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])

    else:
        # Carol ERTHALFelipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ERTHAL Felipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z]) ([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ErthalFelipe Lamarca / Erthal CarolLamarca Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

# upper case
data['Nome Competidor'] = data['Nome Competidor'].str.upper()

# get unique names
names = data['Nome Competidor'].unique()
len(names)

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_20664\221886864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')


1644

In [18]:
# read json to dict
with open('..\data\similar_names_pairs.json', 'r') as fp:
    similar_names_dict = json.load(fp)

# for each key, substitute the values by the key in the data
for key in similar_names_dict:
    for value in similar_names_dict[key]:
        data['Nome Competidor'] = np.where(data['Nome Competidor'] == value, key, data['Nome Competidor'])

In [19]:
names = data['Nome Competidor'].unique()
len(names)

1112

In [20]:
len(data)

48836

In [22]:
# output data to csv
data.to_csv('..\data\Banco de Súmulas - Pair Classes.csv', index=False)